In [1]:
%mkdir MaskRCNN_Defect
%cd MaskRCNN_Defect
!git clone https://github.com/facebookresearch/detectron2.git

/kaggle/working/MaskRCNN_Defect2
Cloning into 'detectron2'...
remote: Enumerating objects: 15860, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 15860 (delta 43), reused 14 (delta 14), pack-reused 15786 (from 4)
Receiving objects: 100% (15860/15860), 6.42 MiB | 20.23 MiB/s, done.
Resolving deltas: 100% (11542/11542), done.


/kaggle/working/MaskRCNN_Defect
Cloning into 'detectron2'...
remote: Enumerating objects: 15860, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 15860 (delta 43), reused 14 (delta 14), pack-reused 15786 (from 4)
Receiving objects: 100% (15860/15860), 6.42 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (11542/11542), done.


In [2]:
#Install requirement packagenya
%cd detectron2
!pip install -e .

/kaggle/working/MaskRCNN_Defect2/detectron2
Obtaining file:///kaggle/working/MaskRCNN_Defect2/detectron2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.0 MB/s eta 0:00:0000:01
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=5a601dce51262c5693e6b6ed556932cb6b63bc1fc5fc0562994d8a8413510caf
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
Successfully built fvcore
  Running setup.py develop for detectron2


In [83]:
#Langsung pindah ke foldernya habis restart session
%cd /kaggle/working/MaskRCNN_Defect/detectron2

/kaggle/working/MaskRCNN_Defect/detectron2


In [85]:
#Import package
import os
import cv2
from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

In [86]:
!cp -r /kaggle/input/hotwheelsdefectcoco /kaggle/working/Dataset

In [87]:

#Register datasetnya
register_coco_instances(
    "train",
    {},
    "/kaggle/working/Dataset/annotations/train_coco_normalized.json",
    "/kaggle/working/Dataset/train"
)
register_coco_instances(
    "val",
    {},
    "/kaggle/working/Dataset/annotations/val_coco_normalized.json",
    "/kaggle/working/Dataset/val"
)
#Di code ini test datasetnya lupa kupake
register_coco_instances(
    "test",
    {},
    "/kaggle/working/Dataset/annotations/test_coco_normalized.json",
    "/kaggle/working/Dataset/test"
)

In [94]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

#Buat confignya, ini config dari gpt
cfg = get_cfg()

# Use a Mask R-CNN model (for polygonal mask output)
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))

# Dataset registration names
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)

cfg.DATALOADER.NUM_WORKERS = 2

# Use pretrained COCO weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

# === Solver ===
cfg.SOLVER.IMS_PER_BATCH = 2                # Smaller batch for low VRAM
cfg.SOLVER.BASE_LR = 0.00025                # Standard learning rate
cfg.SOLVER.MAX_ITER = 4000                  # More iterations for small data
cfg.SOLVER.STEPS = []                       # No LR decay
cfg.SOLVER.CHECKPOINT_PERIOD = 500          # Save every 500 iters

# === ROI HEADS ===
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # Fewer proposals per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3            # Your dataset classes

cfg.TEST.EVAL_PERIOD = 100                  # Evaluate every 500 iters

#Folder output
cfg.OUTPUT_DIR = "./output"
#Buat folder outputnya
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [95]:
from detectron2.engine import DefaultTrainer
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

In [54]:

#Simpen confignya buat dipake di local
file_content = cfg.dump()
with open("/kaggle/working/config.yaml", "w") as file:
    file.write(file_content)

In [69]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import os
import cv2
from matplotlib import pyplot as plt
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo

# Setup config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))  # or whatever config you used
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 👈 change based on your dataset
cfg.MODEL.WEIGHTS ="model_final.pth"  # 👈 your trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1  # set a threshold to filter weak predictions
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # GPU/CPU handling

# Register dummy metadata (for visualization only)
MetadataCatalog.get("test").thing_classes = ["Scratches", "Hole", "Discoloration"]  # 👈 customize to match your dataset

# Create predictor
predictor = DefaultPredictor(cfg)

In [99]:
!zip -r /kaggle/working/MaskRCNN_Defect.zip /kaggle/working/MaskRCNN_Defect

  adding: kaggle/working/MaskRCNN_Defect/ (stored 0%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/ (stored 0%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ (stored 0%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE.md (deflated 20%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/Detectron2-Logo-Horz.svg (deflated 62%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/ (stored 0%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/documentation.md (deflated 44%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/bugs.md (deflated 45%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/config.yml (deflated 45%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/feature-request.md (deflated 48%)
  adding: kaggle/working/MaskRCNN_Defect/detectron2/.github/ISSUE_TEMPLATE/unexpected-problems-bugs.md (deflated